In [7]:
import pyspark

In [25]:
spark = SparkSession.builder.master("local").appName("ER")

In [31]:
import pyspark, time, json, os, sys
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [12]:
start_time = time.clock()

In [33]:
conf = SparkConf().setAppName("er")
sc = SparkContext(conf=conf)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [29]:
type(SparkSession.newSession())

TypeError: newSession() missing 1 required positional argument: 'self'

In [26]:
from pyspark.sql.functions import collect_set

# loading ER model result

deduped_path = "file:///data_mapping/input/dedupe_lucy.csv"
deduped = spark.read.csv(deduped_path, header=True) \
               .withColumnRenamed("Cluster ID","ClusterID")


AttributeError: 'Builder' object has no attribute 'read'

In [34]:

# config

rules = [
    
    # Capacity 1: Middle Name
    {"rule_id": "1", "match":["FIRST_NAME", "LAST_NAME","DATE_OF_BIRTH"], 
         "mismatch": {"MIDDLE_NAMES":{"distance": None}},
         "capability_id": [1], "description": "Mismatch_middle_names1"},
    {"rule_id": "2", "match":["FIRST_NAME", "LAST_NAME","SSN"], 
         "mismatch": {"MIDDLE_NAMES":{"distance": None}},
         "capability_id": [1], "description": "Mismatch_middle_names2"},

    # Capacity 2: Marriage
    {"rule_id": "3", "match":["FIRST_NAME","DATE_OF_BIRTH"],
         "mismatch":{"LAST_NAME":{"distance": None}, },
         "capability_id": [2], "description": "MARRIED/DIVORCED_1"},
    {"rule_id": "4", "match": ["FIRST_NAME", "SSN"], 
         "mismatch": {"LAST_NAME": {"distance": None}, },
         "capability_id": [2], "description": "MARRIED/DIVORCED_2"},
    {"rule_id": "5", "match": ["FIRST_NAME", "DATE_OF_BIRTH"], 
         "mismatch": {"LAST_NAME": {"distance": None},"ADDRESS": {"distance": None} },
         "capability_id": [2], "description": "MARRIED/DIVORCED_ADD_1"},
    {"rule_id": "6", "match": ["FIRST_NAME", "SSN"], 
         "mismatch": {"LAST_NAME": {"distance": None},"ADDRESS": {"distance": None} },
         "capability_id": [2], "description": "MARRIED/DIVORCED_ADD_2"},

    # Capacity 3: Relocation
    {"rule_id": "7", "match": ["FIRST_NAME", "LAST_NAME", "SSN"], 
         "mismatch": {"ADDRESS": {"distance": None}, },
         "capability_id": [3], "description": "RELOCATION_1"},
    {"rule_id": "8", "match": ["FIRST_NAME", "LAST_NAME", "DATE_OF_BIRTH"], 
         "mismatch": {"ADDRESS": {"distance": None}, },
         "capability_id": [3], "description": "RELOCATION_2"},
    {"rule_id": "9", "match": ["FIRST_NAME", "LAST_NAME", "SSN"], 
         "mismatch": {"DL": {"distance": None}, },
         "capability_id": [3], "description": "RELOCATION_DL_1"},
    {"rule_id": "10", "match": ["FIRST_NAME", "LAST_NAME", "DATE_OF_BIRTH"],
         "mismatch": {"DL": {"distance": None}, },
         "capability_id": [3], "description": "RELOCATION_DL_2"},
    
    # capacity 4: Family
    {"rule_id": "11", "match": [ "LAST_NAME", "ADDRESS"], 
         "mismatch": {"FIRST_NAME": {"distance": None}, "DATE_OF_BIRTH": {"distance": None},},
         "capability_id": [4], "description": "FAMILY1"},    
    
    {"rule_id": "12", "match": [ "LAST_NAME", "ADDRESS"], 
         "mismatch": {"FIRST_NAME": {"distance": None}, "SSN": {"distance": None},},
         "capability_id": [4], "description": "FAMILY2"}        
    
        ,{"rule_id": "13", "match": [ "LAST_NAME", "ADDRESS"], 
         "mismatch": {"FIRST_NAME": {"distance": None}, },
         "capability_id": [4], "description": "FAMILY3"}   
    ]


In [5]:
deduped_cols = {
    "ClusterID": {"uniq_val_for_entity": False, "entity_id": True },
    "confidence_score": {"uniq_val_for_entity": True, "entity_id": False},
    "CUSTOMER_ID": {"uniq_val_for_entity": False, "entity_id": False},
    "ORGUNIT_CODE": {"uniq_val_for_entity": False, "entity_id": False},
    "FIRST_NAME": {"uniq_val_for_entity": False, "entity_id": False},
    "MIDDLE_NAMES": {"uniq_val_for_entity": False, "entity_id": False},
    "LAST_NAME": {"uniq_val_for_entity": False, "entity_id": False},
    "DATE_OF_BIRTH": {"uniq_val_for_entity": False, "entity_id": False},
    "ADDRESS": {"uniq_val_for_entity": False, "entity_id": False},
    "ZONE": {"uniq_val_for_entity": False, "entity_id": False},
    "POSTAL_CODE": {"uniq_val_for_entity": False, "entity_id": False},
    "PRIME_BRANCH_ID": {"uniq_val_for_entity": False, "entity_id": False},
    "CUSTOMER_STATUS_CODE": {"uniq_val_for_entity": False, "entity_id": False},
    "City": {"uniq_val_for_entity": False, "entity_id": False},
    "State": {"uniq_val_for_entity": False, "entity_id": False},
    "SSN": {"uniq_val_for_entity": False, "entity_id": False},
    "DL": {"uniq_val_for_entity": False, "entity_id": False}
}


In [6]:

# generate entity table

ids = [col for col in deduped_cols.keys() if deduped_cols[col]["entity_id"] is True]
columns_to_directly_pass = [col for col in deduped_cols.keys() if deduped_cols[col]["uniq_val_for_entity"] is True]
columns_to_count_distinct = [col for col in deduped_cols.keys() if deduped_cols[col]["uniq_val_for_entity"] is False]

deduped.registerTempTable("deduped")
#entity_table_gen_query = "select " + ",".join(ids) + "," + " ,".join(columns_to_directly_pass)+ "," + " ,".join(["count(distinct("+col+")) as "+col+"_count" for col in columns_to_count_distinct]) \
#                         + " from deduped group by " + ",".join(ids)+ "," + " ,".join(columns_to_directly_pass)

entity_table_gen_query = 'select ClusterID,avg(confidence_score),count(distinct(ORGUNIT_CODE)) as ORGUNIT_CODE_count ,count(distinct(City)) as City_count ,count(distinct(FIRST_NAME)) as FIRST_NAME_count ,count(distinct(LAST_NAME)) as LAST_NAME_count ,count(distinct(SSN)) as SSN_count ,count(distinct(ZONE)) as ZONE_count ,count(distinct(PRIME_BRANCH_ID)) as PRIME_BRANCH_ID_count ,count(distinct(ClusterID)) as ClusterID_count ,count(distinct(State)) as State_count ,count(distinct(MIDDLE_NAMES)) as MIDDLE_NAMES_count ,count(distinct(DATE_OF_BIRTH)) as DATE_OF_BIRTH_count ,count(distinct(POSTAL_CODE)) as POSTAL_CODE_count ,count(distinct(ADDRESS)) as ADDRESS_count ,count(distinct(CUSTOMER_STATUS_CODE)) as CUSTOMER_STATUS_CODE_count ,count(distinct(CUSTOMER_ID)) as CUSTOMER_ID_count ,count(distinct(DL)) as DL_count from deduped group by ClusterID'

entity = spark.sql(entity_table_gen_query)

# generate rule_match
conditions = ["case when " + " and ".join([condition for condition in [
                    " and ".join([col+"_count=1" for col in rule["match"]]) if rule['match'] else "",  # exact_match
                    " and ".join([col+"_count>1" for col in rule["mismatch"]]) if rule['mismatch'] else ""  # mismatch
             ] if condition]) + " then 'True' else 'False' end as Rule" + rule["rule_id"] for rule in rules]

entity.registerTempTable("entity")
rule_validation_query =  "select " + ",".join(ids) + " ," + " ,".join(conditions) + " from entity"
entity_match_category = spark.sql(rule_validation_query)


# evaluation metrics

    # exact = spark.read.csv("file:///KIM - ER outcome by rules for name, ssn, dob_INDEXED")
    # deduped = spark.read.csv("file:///opt/output_full_5_10_dupe_only.csv")
    # exact.count()
    # exact
    #
    #
    #
    # deduped.select()# 458499  / 140661
    # deduped.select("Cluster ID").distinct().count() # 208204 / 65073
    # deduped.select("Cluster ID") \
    #        .groupBy("Cluster ID").count() \
    #        .withColumnRenamed("count", "occur_count") \
    #        .groupBy("occur_count").count()\
    #     .show()
    #
    # #


NameError: name 'deduped' is not defined

In [ ]:
entity.filter("Rule1='False'").filter  .show(n=30)

entity_match_category.count() 208,204
not captured by existing rules: 110,950



capacity 1: mismatch middle name: 18,693
capacity 2: married/divorced: 11,159
capacity 3: relocation: 75,825


investigation:
10295: false positive
11820: false positive
15574: diff first name, middle name, same last name, diff address, same zone, same SSN
16320: good match
2136:


In [ ]:
>>> entity_match_category.filter("Rule1=='True' or Rule2=='True'").count()
18693
>>> entity_match_category.filter("Rule3=='True' or Rule4=='True' or Rule5=='True' or Rul   e6=='True'").count()
11159
>>> entity_match_category.filter("Rule7=='True' or Rule8=='True' or Rule9=='True' or Rul   e10=='True'").count()
75825
>>> entity_match_category.filter("Rule11=='True' or Rule12=='True'").count()
1919



In [ ]:
entity_match_category.filter("Rule1=='False'").filter("Rule2=='False'").filter("Rule3=='False'").filter("Rule4=='False'").filter("Rule5=='False'").filter("Rule6=='False'").filter("Rule7=='False'").filter("Rule8=='False'").filter("Rule9=='False'").filter("Rule10=='False'").filter("Rule11=='False'").filter("Rule12=='False'").count()

In [ ]:
entity_table_gen_query = 'select ClusterID,avg(confidence_score),count(distinct(ORGUNIT_CODE)) as ORGUNIT_CODE_count ,count(distinct(City)) as City_count ,count(distinct(FIRST_NAME)) as FIRST_NAME_count ,count(distinct(LAST_NAME)) as LAST_NAME_count ,count(distinct(SSN)) as SSN_count ,count(distinct(ZONE)) as ZONE_count ,count(distinct(PRIME_BRANCH_ID)) as PRIME_BRANCH_ID_count ,count(distinct(ClusterID)) as ClusterID_count ,count(distinct(State)) as State_count ,count(distinct(MIDDLE_NAMES)) as MIDDLE_NAMES_count ,count(distinct(DATE_OF_BIRTH)) as DATE_OF_BIRTH_count ,count(distinct(POSTAL_CODE)) as POSTAL_CODE_count ,count(distinct(ADDRESS)) as ADDRESS_count ,count(distinct(CUSTOMER_STATUS_CODE)) as CUSTOMER_STATUS_CODE_count ,count(distinct(CUSTOMER_ID)) as CUSTOMER_ID_count ,count(distinct(DL)) as DL_count from deduped group by ClusterID'


In [ ]:
>>> deduped.select("ClusterID").groupBy("ClusterID").count().withColumnRenamed("count","occur_count").groupBy("occur_count").count().orderBy("occur_count").show(n=200)
+-----------+------+
|occur_count| count|
+-----------+------+
|          2|185584|
|          3| 16409|
|          4|  3646|
|          5|  1103|
|          6|   493|
|          7|   278|
|          8|   212|
|          9|   138|
|         10|    77|
|         11|    49|
|         12|    16|
|         13|    14|
|         14|    18|
|         15|    17|
|         16|    13|
|         17|     8|
|         18|    14|
|         19|     4|
|         20|     4|
|         21|     5|
|         22|     4|
|         23|     1|
|         24|     5|
|         25|     6|
|         26|     5|
|         27|     1|
|         28|     3|
|         29|     4|
|         30|     8|
|         31|     2|
|         32|     1|
|         33|     4|
|         35|     1|
|         36|     4|
|         37|     2|
|         38|     1|
|         39|     3|
|         40|     1|
|         41|     3|
|         42|     1|
|         43|     4|
|         44|     2|
|         45|     2|
|         46|     2|
|         48|     1|
|         49|     3|
|         50|     1|
|         51|     3|
|         55|     1|
|         56|     1|
|         57|     1|
|         58|     1|
|         59|     1|
|         60|     1|
|         62|     1|
|         69|     1|
|         70|     1|
|         72|     2|
|         79|     1|
|         83|     1|
|         84|     1|
|         94|     1|
|        100|     2|
|        110|     1|
|        116|     1|
|        134|     1|
|        207|     1|
|        256|     1|
|        415|     1|
|       1991|     1|
+-----------+------+


In [ ]:
>>> for col in entity_match_category.columns[1:]:
...     print(col+
... ":"+str(entity_match_category.select(col).filter(col+"='True'").count()))


Results of 4 capacities 

In [ ]:
>>> for col in entity_match_category.columns[1:]:
...     print(col+
... ":"+str(entity_match_category.select(col).filter(col+"='True'").count()))
...
Rule1:17727
Rule2:17813
Rule3:9621
Rule4:9200
Rule5:6364
Rule6:6039
Rule7:68624
Rule8:66369
Rule9:9388
Rule10:9085
Rule11:1700
Rule12:1561

In [ ]:
>>> entity_match_category.filter("Rule1=='False'").filter("Rule2=='False'").filter("Rule3=='False'").filter("Rule4=='False'").filter("Rule5=='False'").filter("Rule6=='False'").filter("Rule7=='False'").filter("Rule8=='False'").filter("Rule9=='False'").filter("Rule10=='False'").filter("Rule11=='False'").filter("Rule12=='False'").count()
109031


In [ ]:
>>> entity_match_category.filter("Rule1=='True' or Rule2=='True'").count()
18693
>>> entity_match_category.filter("Rule3=='True' or Rule4=='True' or Rule5=='True' or Rul   e6=='True'").count()
11159
>>> entity_match_category.filter("Rule7=='True' or Rule8=='True' or Rule9=='True' or Rul   e10=='True'").count()
75825
>>> entity_match_category.filter("Rule11=='True' or Rule12=='True'").count()
1919